# Creating Rotation Series
Here we describe multiple ways to generate a series of scan distorted images.

###### Loads

In [1]:
%matplotlib widget

# This thing just saves some whitespace from a bug in tqdm / notebook
from IPython.display import HTML
display(HTML("""
<style>
.jp-OutputArea-prompt:empty {
  padding: 0;
  border: 0;
}
</style>
"""))

In [2]:
from importlib import reload
import numpy as np
import matplotlib.pyplot as plt
from genSTEM import Model

from ase.spacegroup import crystal
from ase.build import make_supercell, bulk
from ase.visualize.plot import plot_atoms

# Build a model

Build the structure with ase

In [3]:
a=4.0
atoms = bulk('Al', 'fcc', a=a, cubic=True)
atoms = make_supercell(atoms, np.diag([8,8,1]))

plateX = atoms.positions[:,0]==a*8/4
plateY = atoms.positions[:,1]>=a*8/4

plate = plateX&plateY
print(atoms.positions[plate].shape)

atoms.numbers[plate] = 29

fig, ax = plt.subplots()
plot_atoms(atoms)

(12, 3)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

# Build the image with genSTEM

We can create a series of STEM images with or without defects associated with serial image aquisition.

In [4]:
images, scanangles, *_ = Model.get_rotation_series(atoms, vacuum=0,
                                                   pixel_size=0.2, nImages=4, maxScanAngle=360,
                                                   drift_speed=0, drift_angle=0, jitter_strength=0)

ncols = 4 if len(images) > 4 else len(images)
mult = 12 / ncols
nrows = len(images) // ncols if len(images) % ncols == 0 else len(images) // ncols +1
rowcols = np.array([nrows, ncols])

fig, axs = plt.subplots(*rowcols, figsize=mult*rowcols[::-1])
for ax, img, ang in zip(axs.flatten(), images, scanangles):
    ax.imshow(img.get(), cmap='jet')
    ax.text(0,0, ang, va='top', bbox=dict(facecolor='w', alpha=0.7), clip_on=True)
    ax.axis('off')
fig.tight_layout(pad=0, w_pad=0.1, h_pad=0.1)

  0%|          | 0/4 [00:00<?, ?it/s]

Size: 0.000739328 GB
Shape: (4, 152, 152)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

We have multiple ways to add drift to an image.  Since we know the drift is acumulated ofver time we can add drift using a pixels index as time.

In [5]:
images, scanangles, *_ = Model.get_rotation_series(atoms, vacuum=0,
                                                   pixel_size=0.2, nImages=4, maxScanAngle=360,
                                                   drift_speed=12, drift_angle=45, jitter_strength=0,
                                                   centre_drift=True, periodic_boundary=False,
                                                   drift_by_transform=False)

ncols = 4 if len(images) > 4 else len(images)
mult = 12 / ncols
nrows = len(images) // ncols if len(images) % ncols == 0 else len(images) // ncols +1
rowcols = np.array([nrows, ncols])

fig, axs = plt.subplots(*rowcols, figsize=mult*rowcols[::-1])
for ax, img, ang in zip(axs.flatten(), images, scanangles):
    ax.imshow(img.get(), cmap='jet')
    ax.text(0,0, ang, va='top', bbox=dict(facecolor='w', alpha=0.7), clip_on=True)
    ax.axis('off')
fig.tight_layout(pad=0, w_pad=0.1, h_pad=0.1)

  0%|          | 0/4 [00:00<?, ?it/s]

Size: 0.000739328 GB
Shape: (4, 152, 152)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Or, we can add drift using affine transforms.

In [8]:
images, scanangles, *_ = Model.get_rotation_series(atoms, vacuum=0,
                                                   pixel_size=0.2, nImages=4, maxScanAngle=360,
                                                   drift_speed=12, drift_angle=45, jitter_strength=0,
                                                   centre_drift=False, periodic_boundary=False,
                                                   drift_by_transform=True, kwargs_affine={'mode':'constant', 'cval':0})

ncols = 4 if len(images) > 4 else len(images)
mult = 12 / ncols
nrows = len(images) // ncols if len(images) % ncols == 0 else len(images) // ncols +1
rowcols = np.array([nrows, ncols])

fig, axs = plt.subplots(*rowcols, figsize=mult*rowcols[::-1])
for ax, img, ang in zip(axs.flatten(), images, scanangles):
    ax.imshow(img.get(), cmap='jet')
    ax.text(0,0, ang, va='top', bbox=dict(facecolor='w', alpha=0.7), clip_on=True)
    ax.axis('off')
fig.tight_layout(pad=0, w_pad=0.1, h_pad=0.1)

  0%|          | 0/4 [00:00<?, ?it/s]

Size: 0.000739328 GB
Shape: (4, 152, 152)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

To include periodic boundary conditions by tiling the input atoms cell or wrap around from drift.  The later is much faster but can produce image artifacts from boudnary conditions.

In [9]:
images, scanangles, *_ = Model.get_rotation_series(atoms, vacuum=0,
                                                   pixel_size=0.2, nImages=4, maxScanAngle=360,
                                                   drift_speed=12, drift_angle=45, jitter_strength=0,
                                                   centre_drift=True, periodic_boundary=True,
                                                   drift_by_transform=False)

ncols = 4 if len(images) > 4 else len(images)
mult = 12 / ncols
nrows = len(images) // ncols if len(images) % ncols == 0 else len(images) // ncols +1
rowcols = np.array([nrows, ncols])

fig, axs = plt.subplots(*rowcols, figsize=mult*rowcols[::-1])
for ax, img, ang in zip(axs.flatten(), images, scanangles):
    ax.imshow(img.get(), cmap='jet')
    ax.text(0,0, ang, va='top', bbox=dict(facecolor='w', alpha=0.7), clip_on=True)
    ax.axis('off')
fig.tight_layout(pad=0, w_pad=0.1, h_pad=0.1)

  0%|          | 0/4 [00:00<?, ?it/s]

Size: 0.000739328 GB
Shape: (4, 152, 152)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …